In [ ]:
import math
import json
from urllib.parse import unquote_plus
import os 
from io import BytesIO
import time


import sys
from pip._internal import main
main(['install', '-I', '-q', 'boto3', '--target', '/tmp/', '--no-cache-dir', '--disable-pip-version-check'])
sys.path.insert(0,'/tmp/')
import boto3

import awswrangler as wr
import pandas as pd
import PyPDF2




#!/usr/bin/env bash


def lambda_handler(event, context):

    s3 = boto3.client("s3")
    sqs= boto3.client('sqs')
    textract = boto3.client("textract")

    fileObj = s3.get_object(Bucket='annualreport21-22', Key='annual-report-2021-2022.pdf')
    file_content = fileObj["Body"].read()
    pdf = PyPDF2.PdfFileReader(BytesIO(file_content))
    print(len(pdf.pages))
    sizePDF=len(file_content)
    print(sizePDF)
  #####################################Read CSV##############################################
    x=[]
    df = wr.s3.read_csv('s3://annualreport21-22/Queries.csv')
    # print(df['Page '])

    first_page=df.iloc[0,0]
    last_page=df.iloc[-1,0]
    print(first_page)
    print(last_page)
  ####################################Chunk File into Single Pages######################################################3  
    pages=list(df.iloc[:,0].unique())
    
    for i in pages:
        writer = PyPDF2.PdfFileWriter()
        page = pdf.getPage(i-1)
        writer.addPage(page)
        filename='Page'+str(i)+'.pdf'
        os.chdir('/tmp')
        with open(filename, "wb") as x:
            writer.write(x)
        s3.upload_file(filename, 'splitfiles21-22', filename)
        print(filename +" uploaded")
        os.remove(filename)
######################################Return analysis jobs for n pages#############################################################  
    # answers list
    answers=[]
    
    for i in pages:
        filename='Page'+str(i)+'.pdf'
        x=pd.DataFrame(df.loc[df['Page '] == i])
        y=list(x['Query'])
        queries=[]
        for j in y:
            queries.append({"Text":j})
        # print(queries)
        print("----------------------------------------------------------------")
        response = textract.start_document_analysis(
                DocumentLocation={
                    'S3Object': {
                        'Bucket': 'splitfiles21-22',
                        'Name': filename
                    }
                },
                FeatureTypes=[
                    # 'FORMS','TABLES',
                    'QUERIES'],
                QueriesConfig={"Queries": queries},
                OutputConfig={
                    'S3Bucket': 'textract-output-21-22'
                    })
        job_id=response["JobId"]
        
        print("********Textract Response***********")
        print(response)
        ##################################Get Output######################
        time.sleep(30)
        analysis_response = textract.get_document_analysis(JobId=job_id)
        print("********Analysis Response***********")
        time.sleep(30)
        print(analysis_response)
        
        for i in analysis_response['Blocks']:
                if i['BlockType']=='QUERY_RESULT':
                    print(i['Text'])
                    answers.append(i['Text'])
        
        
##################################Store Output Back to CSV#########################################

    df["Answer"]=answers
    wr.s3.to_csv(df, 's3://processed-output-21-22/Query-Answers.csv')